In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

In [ ]:
from transformers import AutoModelForCausalLM
from utils.torch_utils import count_parameters
import torch

model_path="/home/golympie/ai-toolbox/pruned_models/deepseek_coder_v2_lite_instruct_fused_16/"

model=AutoModelForCausalLM.from_pretrained(
    model_path, 
    device_map='cpu',
    trust_remote_code=True
)
count_parameters(model)

# Deepseek-Coder-v2-Lite-Instruct_Fused_2B

sizes

2 = 2B

4 = 2.3B

8 = 3B

16 = 5B

In [ ]:
Deepseek-Coder-V2-Lite-Instruct-Fused-2E-2B-preview-Unhealed
Deepseek-Coder-V2-Lite-Instruct-Fused-4E-2_5B-preview-Unhealed
Deepseek-Coder-V2-Lite-Instruct-Fused-8E-3B-preview-Unhealed
Deepseek-Coder-V2-Lite-Instruct-Fused-16E-5B-preview-Unhealed

In [ ]:
help(model.quantize)

In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer
import torch

model_path = "/home/golympie/ai-toolbox/pruned_models/deepseek_coder_v2_lite_instruct_fused_16/"
quant_path = model_path + "-AWQ"

quant_config = {
    "zero_point": True,
    "q_group_size": 64,
    "w_bit": 4,
    "version": "GEMM",
    # "max_calib_seq_len": 256,
    # "n_parallel_calib_samples":4
}



# Load model
model = AutoAWQForCausalLM.from_pretrained(model_path, device_map="cuda:1")
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# Quantize
model.quantize(
    tokenizer, 
    quant_config=quant_config,
    max_calib_seq_len=256,
    n_parallel_calib_samples=4
)

# Save quantized model
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

print(f'Model is quantized and saved at "{quant_path}"')